In [97]:
from scrape_stats_function import scrape_player_stats
import time 
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
years = ["2024-2025","2023-2024","2022-2023","2021-2022","2020-2021","2019-2020","2018-2019","2017-2018","2016-2017","2015-2016","2014-2015","2013-2014","2012-2013","2011-2012","2010-2011"]
leagueinfo = [
    {
        "name": "Bundesliga",
        "url": "https://fbref.com/en/comps/20/{year}/keepers/{year}-Bundesliga-Stats"
    },
    {
        "name": "Premier League",
        "url": "https://fbref.com/en/comps/9/{year}/keepers/{year}-Premier-League-Stats"
    },
    {
        "name": "La Liga",
        "url": "https://fbref.com/en/comps/12/{year}/keepers/{year}-La-Liga-Stats"
    },
    {
        "name": "Serie A",
        "url": "https://fbref.com/en/comps/11/{year}/keepers/{year}-Serie-A-Stats"
    },
    {
        "name": "Ligue 1",
        "url": "https://fbref.com/en/comps/13/{year}/keepers/{year}-Ligue-1-Stats"
    },
    {
        "name": "Eredivisie",
        "url": "https://fbref.com/en/comps/23/{year}/keepers/{year}-Eredivisie-Stats"
    },
    {
        "name": "Primeira Liga",
        "url": "https://fbref.com/en/comps/32/{year}/keepers/{year}-Primeira-Liga-Stats"
    },
    {
        "name": "EFL Championship",
        "url": "https://fbref.com/en/comps/10/{year}/keepers/{year}-EFL-Championship-Stats"
    },
    {
        "name":"Scottish Premiership",
        "url":"https://fbref.com/en/comps/40/{year}/keepers/{year}-Scottish-Premiership-Stats"
    }
]

In [99]:
leagues = []
for year in years:
    for league in leagueinfo:
        leagues.append({
            "name": f"{league['name']} {year}",
            "url": league["url"].format(year=year),
        })

league_dfs = {}
combined_list = []
for league in leagues:
    df = scrape_player_stats(league["name"], league["url"], table_id='stats_keeper')
    if df is not None:
        league_dfs[league["name"]] = df
        combined_list.append(df)
    else:
        print(f"Failed for {league['name']} at {league['url']}")
    time.sleep(5)  

200 https://fbref.com/en/comps/20/2024-2025/keepers/2024-2025-Bundesliga-Stats
Found stats_keeper in comment for Bundesliga 2024-2025
200 https://fbref.com/en/comps/9/2024-2025/keepers/2024-2025-Premier-League-Stats
Found stats_keeper in comment for Premier League 2024-2025
200 https://fbref.com/en/comps/12/2024-2025/keepers/2024-2025-La-Liga-Stats
Found stats_keeper in comment for La Liga 2024-2025
200 https://fbref.com/en/comps/11/2024-2025/keepers/2024-2025-Serie-A-Stats
Found stats_keeper in comment for Serie A 2024-2025
200 https://fbref.com/en/comps/13/2024-2025/keepers/2024-2025-Ligue-1-Stats
Found stats_keeper in comment for Ligue 1 2024-2025
200 https://fbref.com/en/comps/23/2024-2025/keepers/2024-2025-Eredivisie-Stats
Found stats_keeper in comment for Eredivisie 2024-2025
200 https://fbref.com/en/comps/32/2024-2025/keepers/2024-2025-Primeira-Liga-Stats
Found stats_keeper in comment for Primeira Liga 2024-2025
200 https://fbref.com/en/comps/10/2024-2025/keepers/2024-2025-EFL-C

In [234]:
df2 = pd.concat(combined_list,ignore_index=True)


df2 = df2[~df2['Player'].str.contains("Player", case=False, na=False)]
df2 = df2.drop(['Matches','Season','Rk'], axis=1)
df2.columns

Index(['Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts',
       'Min', '90s', 'GA', 'GA90', 'SoTA', 'Saves', 'Save%', 'W', 'D', 'L',
       'CS', 'CS%', 'PKatt', 'PKA', 'PKsv', 'PKm', 'Save%.1', 'League'],
      dtype='object')

In [235]:
int64_col = ['Born','Age','MP','Starts','SoTA','W','D','L','CS','PKatt','PKA','PKsv','PKm','GA','Min','Saves']
float64_col = ['90s','GA90','Save%','CS%','Save%.1']


for col in float64_col:
    df2[col] = pd.to_numeric(df2[col], errors='coerce')

for col in int64_col:
    df2[col] = pd.to_numeric(df2[col], errors='coerce').astype('Int64')

df2 = df2.astype({
    'Player': 'string',
    'Nation': 'string',
    'Pos': 'string',
    'Squad': 'string',
    'League': 'string',
})



In [236]:
df2.dtypes

Player     string[python]
Nation     string[python]
Pos        string[python]
Squad      string[python]
Age                 Int64
Born                Int64
MP                  Int64
Starts              Int64
Min                 Int64
90s               float64
GA                  Int64
GA90              float64
SoTA                Int64
Saves               Int64
Save%             float64
W                   Int64
D                   Int64
L                   Int64
CS                  Int64
CS%               float64
PKatt               Int64
PKA                 Int64
PKsv                Int64
PKm                 Int64
Save%.1           float64
League     string[python]
dtype: object

In [228]:
df2['PKA'].unique()

<IntegerArray>
[0, 2, 3, 4, 6, 1, 5, 9, 7, 8, 13, <NA>, 10, 11, 14]
Length: 15, dtype: Int64

In [ ]:
pd.set_option('display.max_columns', None)


new_columns = {
    'MP': 'Matches Played',
    'GA': 'Goals Against',
    'GA90': 'Goals Aginst/90',
    'SoTA': 'Shots on Target Against',
    'Save%': 'Save %',
    'CS': 'Clean Sheets',
    'CS%': 'Clean Sheet %',
    'PKatt': 'Pen Kicks Attempted',
    'PKA': 'Pen Kicks Allowed',
    'PKsv': 'Pen Kicks Saved',
    'PKm': 'Pen Kicks Missed',
    'Save%.1': 'Pen Save %',
}
df2.rename(columns=new_columns, inplace=True)
df2

,Player,Nation,Pos,Squad,Age,Born,Matches Played,Starts,Min,90s,Goals Against,Goals Aginst/90,Shots on Target Against,Saves,Save Percentage,W,D,L,Clean Sheets,Clean Sheet Percentage,Pen Kicks Attempted,Pen Kicks Allowed,Pen Kicks Saved,Pen Kicks Missed,Pen Save Percentage,League
0,Ben Alexander Voll,de GER,GK,St. Pauli,23,2000,2,1,91,1.0,2,1.98,5,3,60.0,0,1,0,0,0.0,0,0,0,0,NaN,Bundesliga 2024-2025
1,Noah Atubolu,de GER,GK,Freiburg,22,2002,26,26,2307,25.6,37,1.44,120,81,69.2,12,6,8,10,38.5,2,0,2,0,100.0,Bundesliga 2024-2025
2,Oliver Baumann,de GER,GK,Hoffenheim,34,1990,28,28,2520,28.0,53,1.89,138,84,63.0,6,8,14,4,14.3,5,2,3,0,60.0,Bundesliga 2024-2025
3,Tiago Cardoso,lu LUX,GK,Gladbach,18,2006,5,4,380,4.2,3,0.71,23,20,87.0,2,1,1,2,50.0,0,0,0,0,NaN,Bundesliga 2024-2025
4,Finn Dahmen,de GER,GK,Augsburg,26,1998,19,19,1710,19.0,19,1.00,82,65,79.3,7,6,6,9,47.4,2,2,0,0,0.0,Bundesliga 2024-2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3824,Ryan Scully,sct SCO,GK,Partick Thistle,22,1992,10,9,863,9.6,19,1.98,46,27,58.7,3,3,4,1,11.1,<NA>,<NA>,<NA>,<NA>,NaN,Scottish Premiership 2015-2016
3825,Michał Szromnik,pl POL,GK,Dundee United,22,1993,9,9,810,9.0,15,1.67,36,21,58.3,1,2,6,1,11.1,<NA>,<NA>,<NA>,<NA>,NaN,Scottish Premiership 2015-2016
3826,Danny Ward,wls WAL,GK,Aberdeen,22,1993,21,21,1890,21.0,23,1.10,85,62,72.9,13,4,4,8,38.1,<NA>,<NA>,<NA>,<NA>,NaN,Scottish Premiership 2015-2016
3828,Gary Woods,eng ENG,GK,Ross County,24,1990,13,11,1065,11.8,22,1.86,60,38,63.3,4,1,7,3,27.3,<NA>,<NA>,<NA>,<NA>,NaN,Scottish Premiership 2015-2016


In [232]:
df2.isnull().sum()

Player                        0
Nation                        2
Pos                           2
Squad                         0
Age                           3
Born                          3
Matches Played                0
Starts                        0
Min                           0
90s                           0
Goals Against                11
Goals Aginst/90              14
Shots on Target Against      38
Saves                        38
Save Percentage              97
W                             0
D                             0
L                             0
Clean Sheets                 14
Clean Sheet Percentage      167
Pen Kicks Attempted         410
Pen Kicks Allowed           409
Pen Kicks Saved             409
Pen Kicks Missed            410
Pen Save Percentage        1397
League                        0
dtype: int64

In [233]:
df2['L']= df2['L'].fillna(0)
df2['D']= df2['D'].fillna(0)
df2['W']= df2['W'].fillna(0)
df2['Min']= df2['Min'].fillna(0)
